In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
from Utils import helper as hp
warnings.filterwarnings('ignore')
import pyarrow.json as pj
import json
import pyarrow.parquet as pq
import ast

## EDA review.json

In [2]:
df = pd.DataFrame()

In [3]:
lines_json = []
with open('../TestData/Yelp/review.json', 'r', encoding='utf-8') as f:
    # Iterate over each line in the JSON file
    for line in f:
        # Parse the JSON string into a dictionary
        data = json.loads(line)
        lines_json.append(data)

reviews_df = pd.DataFrame(lines_json)

In [4]:
# Vista Preliminar de Datos

print(reviews_df.info())
reviews_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6990280 entries, 0 to 6990279
Data columns (total 9 columns):
 #   Column       Dtype  
---  ------       -----  
 0   review_id    object 
 1   user_id      object 
 2   business_id  object 
 3   stars        float64
 4   useful       int64  
 5   funny        int64  
 6   cool         int64  
 7   text         object 
 8   date         object 
dtypes: float64(1), int64(3), object(5)
memory usage: 480.0+ MB
None


,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5.0,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3.0,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15


In [5]:
# Reivsión de Nulos

reviews_overview = hp.data_overview(reviews_df)
reviews_overview

,nombre_campo,tipo_datos,NO_nulos_%,nulos_%,nulos_cantidad
0,review_id,[<class 'str'>],100.0,0.0,0
1,user_id,[<class 'str'>],100.0,0.0,0
2,business_id,[<class 'str'>],100.0,0.0,0
3,stars,[<class 'float'>],100.0,0.0,0
4,useful,[<class 'int'>],100.0,0.0,0
5,funny,[<class 'int'>],100.0,0.0,0
6,cool,[<class 'int'>],100.0,0.0,0
7,text,[<class 'str'>],100.0,0.0,0
8,date,[<class 'str'>],100.0,0.0,0


In [8]:
# Revisión de duplicados

reviews_df[reviews_df.duplicated()]

,review_id,user_id,business_id,stars,useful,funny,cool,text,date


In [6]:
# Revisión de Outliers
outliers_columns = ['stars','useful','funny','cool']

for column in outliers_columns:
    hp.outliers_percentage(reviews_df, column)
    print("-"*66)

Total de outliers en stars: 0 de 6990280 registros -- 0.0 %
------------------------------------------------------------------
Total de outliers en useful: 83325 de 6990280 registros -- 1.19 %
------------------------------------------------------------------
Total de outliers en funny: 59789 de 6990280 registros -- 0.86 %
------------------------------------------------------------------
Total de outliers en cool: 58316 de 6990280 registros -- 0.83 %
------------------------------------------------------------------


In [7]:
# Parquet - Archivo Final Tips
reviews_df_ruta = '../Yelp-Data/review.parquet'
reviews_df.to_parquet(reviews_df_ruta, index=False)

## EDA del archivo user.parquet 

In [4]:
user_parquet = pq.ParquetFile(r'data-states\user.parquet')
for batch in user_parquet.iter_batches():
    batch_df = batch.to_pandas()

In [5]:
hp.data_overview(batch_df)

,nombre_campo,tipo_datos,NO_nulos_%,nulos_%,nulos_cantidad
0,user_id,[<class 'str'>],100.0,0.0,0
1,name,[<class 'str'>],100.0,0.0,0
2,review_count,[<class 'int'>],100.0,0.0,0
3,yelping_since,[<class 'str'>],100.0,0.0,0
4,useful,[<class 'int'>],100.0,0.0,0
5,funny,[<class 'int'>],100.0,0.0,0
6,cool,[<class 'int'>],100.0,0.0,0
7,elite,[<class 'str'>],100.0,0.0,0
8,friends,[<class 'str'>],100.0,0.0,0
9,fans,[<class 'int'>],100.0,0.0,0


In [6]:
batch_df.isna().sum()

user_id               0
name                  0
review_count          0
yelping_since         0
useful                0
funny                 0
cool                  0
elite                 0
friends               0
fans                  0
average_stars         0
compliment_hot        0
compliment_more       0
compliment_profile    0
compliment_cute       0
compliment_list       0
compliment_note       0
compliment_plain      0
compliment_cool       0
compliment_funny      0
compliment_writer     0
compliment_photos     0
dtype: int64

In [7]:
batch_df[batch_df.duplicated()]

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos


In [8]:
outliers_columns = ["useful","funny","cool",'fans',"average_stars"]

for column in outliers_columns:
    hp.outliers_percentage(batch_df, column)
    print("-"*66)

Total de outliers en useful: 91 de 8445 registros -- 1.08 %
------------------------------------------------------------------
Total de outliers en funny: 79 de 8445 registros -- 0.94 %
------------------------------------------------------------------
Total de outliers en cool: 71 de 8445 registros -- 0.84 %
------------------------------------------------------------------
Total de outliers en fans: 146 de 8445 registros -- 1.73 %
------------------------------------------------------------------
Total de outliers en average_stars: 200 de 8445 registros -- 2.37 %
------------------------------------------------------------------


## EDA business.pkl

In [8]:
business_df = pd.read_pickle("../TestData/Yelp/business.pkl")

In [9]:
business_df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,NaN,93101,34.426679,-119.711197,5.0,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,NaN,63123,38.551126,-90.335695,3.0,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,NaN,85711,32.223236,-110.880452,3.5,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,MO,18054,40.338183,-75.471659,4.5,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# Se realiza una máscara del df, ya que se encuentra con columnas duplicadas
business_df_mask = business_df.iloc[:, :14].copy()
business_df_mask.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,NaN,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,NaN,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,NaN,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,MO,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


In [11]:
# Vista Preliminar de Datos
business_df_mask.info()

<class 'pandas.core.frame.DataFrame'>
Index: 150346 entries, 0 to 150345
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   business_id   150346 non-null  object
 1   name          150346 non-null  object
 2   address       150346 non-null  object
 3   city          150346 non-null  object
 4   state         150343 non-null  object
 5   postal_code   150346 non-null  object
 6   latitude      150346 non-null  object
 7   longitude     150346 non-null  object
 8   stars         150346 non-null  object
 9   review_count  150346 non-null  object
 10  is_open       150346 non-null  object
 11  attributes    136602 non-null  object
 12  categories    150243 non-null  object
 13  hours         127123 non-null  object
dtypes: object(14)
memory usage: 17.2+ MB


In [12]:
# Revisión de Nulos
business_overview = hp.data_overview(business_df_mask)
business_overview

,nombre_campo,tipo_datos,NO_nulos_%,nulos_%,nulos_cantidad
0,business_id,[<class 'str'>],100.00,0.00,0
1,name,[<class 'str'>],100.00,0.00,0
2,address,[<class 'str'>],100.00,0.00,0
3,city,[<class 'str'>],100.00,0.00,0
4,state,"[<class 'float'>, <class 'str'>]",100.00,0.00,3
5,postal_code,[<class 'str'>],100.00,0.00,0
6,latitude,[<class 'float'>],100.00,0.00,0
7,longitude,[<class 'float'>],100.00,0.00,0
8,stars,[<class 'float'>],100.00,0.00,0
9,review_count,[<class 'int'>],100.00,0.00,0


In [13]:
# Revisión de Duplicados
business_duplicated = business_df_mask[['business_id', 'name', 'address', 'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars', 'review_count', 'is_open', 'categories']]
business_duplicated[business_duplicated.duplicated()]

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories


In [14]:
# Revisión de Outliers
outliers_columns = ['stars', 'review_count', 'is_open']

for column in outliers_columns:
    hp.outliers_percentage(business_df_mask, column)
    print("-"*66)

Total de outliers en stars: 0 de 150346 registros -- 0.0 %
------------------------------------------------------------------
Total de outliers en review_count: 1942 de 150346 registros -- 1.29 %
------------------------------------------------------------------
Total de outliers en is_open: 0 de 150346 registros -- 0.0 %
------------------------------------------------------------------


## EDA del archivo tip.json 

In [17]:
df_tip_ruta = '../TestData/Yelp/tip.json'

# Lista para almacenar los diccionarios JSON de cada línea
data_list = []

# Abrir el archivo y procesar cada línea
with open(df_tip_ruta, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            # Usar ast.literal_eval para convertir la línea en un diccionario
            json_data = ast.literal_eval(line)
            data_list.append(json_data)
        except ValueError as e:
            print(f"Error en la línea: {line}")
            continue

# Crear un DataFrame a partir de la lista de diccionarios
df_tip = pd.DataFrame(data_list)

In [18]:
df_tip

,user_id,business_id,text,date,compliment_count
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05 18:35:10,0
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,2013-08-18 00:56:08,0
3,FjMQVZjSqY8syIO-53KFKw,hV-bABTK-glh5wj31ps_Jw,Very decent fried chicken,2017-06-27 23:05:38,0
4,ld0AperBXk1h6UbqmM80zw,_uN0OudeJ3Zl_tf6nxg5ww,Appetizers.. platter special for lunch,2012-10-06 19:43:09,0
...,...,...,...,...,...
908910,eYodOTF8pkqKPzHkcxZs-Q,3lHTewuKFt5IImbXJoFeDQ,Disappointed in one of your managers.,2021-09-11 19:18:57,0
908911,1uxtQAuJ2T5Xwa_wp7kUnA,OaGf0Dp56ARhQwIDT90w_g,Great food and service.,2021-10-30 11:54:36,0
908912,v48Spe6WEpqehsF2xQADpg,hYnMeAO77RGyTtIzUSKYzQ,Love their Cubans!!,2021-11-05 13:18:56,0
908913,ckqKGM2hl7I9Chp5IpAhkw,s2eyoTuJrcP7I_XyjdhUHQ,Great pizza great price,2021-11-20 16:11:44,0


In [19]:
hp.data_overview(df_tip)

,nombre_campo,tipo_datos,NO_nulos_%,nulos_%,nulos_cantidad
0,user_id,[<class 'str'>],100.0,0.0,0
1,business_id,[<class 'str'>],100.0,0.0,0
2,text,[<class 'str'>],100.0,0.0,0
3,date,[<class 'str'>],100.0,0.0,0
4,compliment_count,[<class 'int'>],100.0,0.0,0


In [ ]:
df_tip[df_tip.duplicated()]

No hay alguna columna a la cual sea necesario calcularle los outliers, así que el EDA de calidad <br>
de dato para este dataframe de Tip, por este primer sprint permanece así 

In [ ]:
# Parquet - Archivo Final Tips
df_tips_ruta = '../Yelp-Data/tip.parquet'
df_tip.to_parquet(df_tips_ruta, index=False)